In [41]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder


In [42]:
mimicDF = pd.read_csv('../LEN_Test/data/StaticData.csv')
mimicDF = mimicDF.drop(columns=['PatientID', 'deathperiod'])

In [43]:
mimicDF.head()

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
0,9,1,74.19,7,7,0.0,1.0,NaN,0.0,3.0,3.0
1,2,0,75.00,4,2,NaN,0.0,0.0,1.0,NaN,1.0
2,16,0,51.92,4,2,NaN,0.0,NaN,0.0,NaN,2.0
3,12,0,51.73,-13,4,2.0,2.0,NaN,0.0,NaN,0.0
4,7,0,62.07,0,3,0.0,2.0,NaN,1.0,NaN,0.0


In [44]:
mimicDF.describe()

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
count,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,3311.000000,5227.000000,2254.000000,5261.000000,1294.000000,5212.000000
mean,14.668757,0.382554,75.049002,9.801026,4.794185,1.987315,0.674957,0.696983,1.017297,3.238022,0.768611
std,14.101457,0.486057,50.553676,9.671438,2.694694,1.374066,0.892066,1.051290,1.263514,0.706116,1.140061
min,1.000000,0.000000,16.010000,-16.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,0.000000,55.410000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,10.000000,0.000000,68.300000,9.000000,4.000000,2.000000,0.000000,0.000000,1.000000,3.000000,0.000000
75%,18.000000,1.000000,78.525000,16.000000,6.000000,3.000000,1.000000,1.000000,2.000000,4.000000,1.000000
max,202.000000,1.000000,306.880000,47.000000,22.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [45]:
rowsWithNaN = sum(mimicDF.isnull().any(axis=1))
print(f"{mimicDF.shape[0]} rows in df, {rowsWithNaN} containing NaN values")

5262 rows in df, 4778 containing NaN values


In [46]:
for col in mimicDF:
    mimicDF[col] = mimicDF[col].fillna(mimicDF[col].mean())

In [47]:
mimicDF

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
0,9,1,74.19,7,7,0.000000,1.0,0.696983,0.0,3.000000,3.0
1,2,0,75.00,4,2,1.987315,0.0,0.000000,1.0,3.238022,1.0
2,16,0,51.92,4,2,1.987315,0.0,0.696983,0.0,3.238022,2.0
3,12,0,51.73,-13,4,2.000000,2.0,0.696983,0.0,3.238022,0.0
4,7,0,62.07,0,3,0.000000,2.0,0.696983,1.0,3.238022,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5257,120,1,36.61,5,10,3.000000,3.0,4.000000,0.0,3.238022,0.0
5258,13,0,53.90,12,4,3.000000,0.0,0.696983,0.0,3.238022,1.0
5259,4,0,58.93,18,5,1.987315,1.0,0.000000,4.0,3.238022,0.0
5260,16,0,68.98,-4,3,1.000000,2.0,0.696983,0.0,3.238022,0.0


In [48]:
mimicDF.describe()

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
count,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000
mean,14.668757,0.382554,75.049002,9.801026,4.794185,1.987315,0.674957,0.696983,1.017297,3.238022,0.768611
std,14.101457,0.486057,50.553676,9.671438,2.694694,1.089902,0.889093,0.687969,1.263394,0.350059,1.134631
min,1.000000,0.000000,16.010000,-16.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,0.000000,55.410000,2.000000,3.000000,1.987315,0.000000,0.000000,0.000000,3.238022,0.000000
50%,10.000000,0.000000,68.300000,9.000000,4.000000,1.987315,0.000000,0.696983,1.000000,3.238022,0.000000
75%,18.000000,1.000000,78.525000,16.000000,6.000000,3.000000,1.000000,0.696983,2.000000,3.238022,1.000000
max,202.000000,1.000000,306.880000,47.000000,22.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [49]:
rowsWithNaN = sum(mimicDF.isnull().any(axis=1))
print(f"{mimicDF.shape[0]} rows in df, {rowsWithNaN} containing NaN values")

5262 rows in df, 0 containing NaN values


#### Replaced NaN values with average of respective columns

In [50]:
bins = 5

est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='uniform')
categorisedDF = pd.DataFrame(data=est.fit_transform(mimicDF), columns=mimicDF.columns)

In [51]:
categorisedDF

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
0,0.0,4.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,3.0,3.0
1,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,4.0,1.0
2,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,4.0,2.0
3,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,4.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5257,2.0,4.0,0.0,1.0,2.0,3.0,3.0,4.0,0.0,4.0,0.0
5258,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,4.0,1.0
5259,0.0,0.0,0.0,2.0,0.0,2.0,1.0,0.0,4.0,4.0,0.0
5260,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,4.0,0.0


In [52]:
categorisedDF.describe()

,los,gender,age,comorbidity,sofa,respiration,coagulation,liver,renal,cardiovascular,cns
count,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000
mean,0.049601,1.530217,0.497529,1.554922,0.385405,1.992018,0.670468,0.298556,1.017294,3.812619,0.761307
std,0.256471,1.944228,0.880146,0.790029,0.643184,1.089919,0.890785,0.769595,1.263394,0.479823,1.137079
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,4.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,0.000000,1.000000,4.000000,0.000000
75%,0.000000,4.000000,1.000000,2.000000,1.000000,3.000000,1.000000,0.000000,2.000000,4.000000,1.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [53]:
categories = {0: 'very low', 1: 'low', 2: 'medium', 3: 'high', 4: 'very high'}

for col in categorisedDF.columns:
    categorisedDF[col] = categorisedDF[col].map(categories)

categorisedDF = pd.get_dummies(categorisedDF)

categorisedDF.head()

,los_high,los_low,los_medium,los_very high,los_very low,gender_very high,gender_very low,age_low,age_very high,age_very low,...,renal_very low,cardiovascular_high,cardiovascular_medium,cardiovascular_very high,cardiovascular_very low,cns_high,cns_low,cns_medium,cns_very high,cns_very low
0,0,0,0,0,1,1,0,1,0,0,...,1,1,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,0,0,0,1,0,1,0,0,1,...,1,0,0,1,0,0,0,1,0,0
3,0,0,0,0,1,0,1,0,0,1,...,1,0,0,1,0,0,0,0,0,1
4,0,0,0,0,1,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1


In [54]:
categorisedDF.describe()

,los_high,los_low,los_medium,los_very high,los_very low,gender_very high,gender_very low,age_low,age_very high,age_very low,...,renal_very low,cardiovascular_high,cardiovascular_medium,cardiovascular_very high,cardiovascular_very low,cns_high,cns_low,cns_medium,cns_very high,cns_very low
count,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,...,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000,5262.000000
mean,0.000190,0.037628,0.004181,0.000760,0.957241,0.382554,0.617446,0.322691,0.043710,0.633599,...,0.474914,0.110984,0.037818,0.851007,0.000190,0.075447,0.197453,0.088559,0.040099,0.598442
std,0.013786,0.190314,0.064531,0.027563,0.202333,0.486057,0.486057,0.467550,0.204468,0.481867,...,0.499418,0.314143,0.190775,0.356115,0.013786,0.264136,0.398115,0.284134,0.196210,0.490260
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
x_train = torch.tensor(categorisedDF.to_numpy(), dtype=torch.float)


### TODO Do we want to standardise the data before categorising?

If another point (eg an outlier) is added to the dataset, then this could push the other points away from their original categorisation. Need outlier detection? 